# ДЗ 4

* Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) сверточной сетью (самописной) - **сделано**

* Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50
* Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50 с аугментацией (самописной, с использованием Pytorch встроенных методов)

# Извините, еще в процессе, будет закончено до конца дня 20.02


In [1]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True)

Files already downloaded and verified


In [3]:
class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]
    
trans_actions = transforms.Compose([transforms.Scale(44),
                                    transforms.RandomCrop(32, padding=4), 
                                    transforms.ToTensor()])

train_dataset, valid_dataset = train_test_split(dataset, test_size=0.05, random_state=13)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [12]:
def evaluate(net, ds):
    dl = torch.utils.data.DataLoader(ds, batch_size=128, shuffle=False)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.eval()
    total_loss = 0
    count = 0
    matches = 0
    for X, y in dl:
        preds = net(X)
        total_loss += loss(preds, y)
        pred_classes = torch.argmax(preds, dim=1)
        matches += int(sum(pred_classes == y))
        count += len(y)

    return float(total_loss/count), float(matches/count), count
 
def train_and_test(net, ds_train, ds_test, optimizer_class=torch.optim.Adam, n_epochs=10, lr=0.01, report_on=-1, batch_size=1024):
    
    print('========================================================')
    print(' Обучаем сеть: ', net)
    
    optimizer = optimizer_class(net.parameters(), lr=0.01)
    print(' Оптимизатор: ', optimizer)

    def print_results():
        for title, ds in [('Тренировочный', ds_train), ('Валидационный', ds_test)]:
            r = evaluate(net, ds)
            print(f"    {title} набор: кросс-энтропия: {r[0]:.2f}, доля совпадений: {r[1]*100:.1f}%")
    print(' До обучения')
    print_results()
    
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()    

    net.train()
    report_on = max(len(dl_train) if report_on == -1 else report_on, 1)
    for epoch in range(n_epochs):
        print('--------- Эпоха ', epoch, '/', n_epochs, ' ------------------')
        running_loss = 0.0
        for i, data in enumerate(tqdm(dl_train)):
            inputs, labels = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if (i+1) % report_on == 0:    # печатаем каждые report_on mini-batches
                print('Выучено батчей : %5d; loss: %.3f' % (i + 1, running_loss / report_on))
                running_loss = 0.0
                
    print(' Обучение закончено!')
    print(' После обучения')
    print_results()


In [8]:
class Net(nn.Module):

    def __init__(self, output_classes=100):
        super(Net, self).__init__()
        self.dp_three = nn.Dropout(0.2)
        self.dp_four = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, 3)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 60, 3)
        self.bn_three = torch.nn.BatchNorm2d(60)
        self.conv_three = torch.nn.Conv2d(60, 120, 3)
        self.bn_four = torch.nn.BatchNorm2d(120)
        self.fc1 = torch.nn.Linear(480, 200)
        self.fc2 = torch.nn.Linear(200, 60)
        self.out = torch.nn.Linear(60, output_classes)
        
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_three(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_four(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.out(x)
       
net = Net()
print(net)

Net(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=100, bias=True)
)


In [13]:
for i in range(5):
    train_and_test(net, train_dataset, valid_dataset, n_epochs=5) #5 по 5 эпох с выводом промежуточного результата

 Обучаем сеть:  Net(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=100, bias=True)
)
 Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
 До обуч

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:46<00:00,  1.02it/s]


Выучено батчей :    47; loss: 3.219
--------- Эпоха  1 / 5  ------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:45<00:00,  1.03it/s]


Выучено батчей :    47; loss: 3.099
--------- Эпоха  2 / 5  ------------------


...

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:45<00:00,  1.04it/s]


Выучено батчей :    47; loss: 2.748
--------- Эпоха  4 / 5  ------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:45<00:00,  1.04it/s]

Выучено батчей :    47; loss: 2.738
 Обучение закончено!
 После обучения


    Тренировочный набор: кросс-энтропия: 2.56, доля совпадений: 34.1%
    Валидационный набор: кросс-энтропия: 3.28, доля совпадений: 24.4%
